<a href="https://colab.research.google.com/github/MinGyeongdo/Sesion2_projcet/blob/main/Model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.2 MB/s 


In [1]:
import pandas as pd

from google.colab import files

uploaded = files.upload()

Saving y_val_log.csv to y_val_log.csv
Saving y_train_log.csv to y_train_log.csv
Saving y_test.csv to y_test.csv
Saving X_val.csv to X_val.csv
Saving X_train.csv to X_train.csv
Saving X_test.csv to X_test.csv


In [21]:
# 사용한 라이브러리
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 

import numpy as np
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from scipy.stats import norm
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.impute import SimpleImputer
from scipy.stats import randint, uniform

In [16]:
X_test = pd.read_csv('X_test.csv').drop('Unnamed: 0', axis='columns')
y_test = pd.read_csv('y_test.csv').drop('Unnamed: 0', axis='columns')
X_train = pd.read_csv('X_train.csv').drop('Unnamed: 0', axis='columns')
y_train_log = pd.read_csv('y_train_log.csv').drop('Unnamed: 0', axis='columns')
X_val = pd.read_csv('X_val.csv').drop('Unnamed: 0', axis='columns')
y_val_log = pd.read_csv('y_val_log.csv').drop('Unnamed: 0', axis='columns')

In [17]:
X_train.head()

,rank,titles,years,imdp_ratings,meta_scores,votes,directors,movie_rates,genre1,genre2,lead_actor,actor1
0,43204,The Grand Budapest Hotel,2014,8.1,88.0,790805.0,WesAnderson,15,Adventure,Comedy,RalphFiennes,F.MurrayAbraham
1,12150,Inglourious Basterds,2009,8.3,69.0,1410621.0,QuentinTarantino,15,Adventure,Drama,BradPitt,DianeKruger
2,48626,Captain Marvel,2019,6.8,64.0,543460.0,sAnnaBodenRyanFleck,12,Action,Adventure,BrieLarson,SamuelL.Jackson
3,30676,The Avengers,2012,8.0,69.0,1367269.0,JossWhedon,12,Action,Adventure,RobertDowneyJr.,ChrisEvans
4,14080,Guardians of the Galaxy,2014,8.0,76.0,1153235.0,JamesGunn,12,Action,Adventure,ChrisPratt,VinDiesel


In [18]:
# XGBRegressor

pipe_XGB = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    XGBRegressor(
    n_estimator=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    #learning_rate = 0.1,
    scale_pos_weight=1,
    seed=42,
    n_jobs= -1)  
)
pipe_XGB.fit(X_train, y_train_log)

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['titles', 'directors', 'movie_rates',
                                     'genre1', 'genre2', 'lead_actor',
                                     'actor1'])),
                ('simpleimputer', SimpleImputer()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.8, max_depth=5,
                              n_estimator=1000, n_jobs=-1,
                              objective='reg:squarederror', seed=42,
                              subsample=0.8))])

In [20]:
pipe_XGB.score(X_val, y_val_log)

0.9161673905376794

In [22]:
dists = {
    'xgbregressor__learning_rate': [0.1, 0.2, 0.08, 0.05], 
    }

clf = RandomizedSearchCV(
    pipe_XGB, 
    param_distributions=dists, 
    n_iter=25, 
    cv=5, 
    scoring='r2',  
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train_log);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 4 is smaller than n_iter=25. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


KeyboardInterrupt: ignored